# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 8:30AM,242696,10,0085905299,ISDIN Corporation,Released
1,Jul 26 2022 8:30AM,242699,10,0085905304,ISDIN Corporation,Released
2,Jul 26 2022 8:30AM,242699,10,0085905302,ISDIN Corporation,Released
3,Jul 26 2022 8:30AM,242696,10,0085905300,ISDIN Corporation,Released
4,Jul 26 2022 8:30AM,242696,10,0085905298,ISDIN Corporation,Released
5,Jul 26 2022 8:30AM,242696,10,0085905301,ISDIN Corporation,Released
6,Jul 26 2022 8:30AM,242699,10,0085905375,ISDIN Corporation,Released
7,Jul 26 2022 8:30AM,242696,10,0085905378,ISDIN Corporation,Released
8,Jul 26 2022 8:30AM,242696,10,0085905377,ISDIN Corporation,Released
9,Jul 26 2022 8:30AM,242696,10,0085905417,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,242695,Executing,2
26,242696,Released,53
27,242697,Released,1
28,242699,Released,17
29,242701,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242695,2.0,NaN
242696,NaN,53.0
242697,NaN,1.0
242699,NaN,17.0
242701,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242576,0.0,1.0
242579,0.0,1.0
242581,0.0,1.0
242583,0.0,1.0
242612,0.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242576,0,1
242579,0,1
242581,0,1
242583,0,1
242612,0,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242576,0,1
1,242579,0,1
2,242581,0,1
3,242583,0,1
4,242612,0,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242576,,1
1,242579,,1
2,242581,,1
3,242583,,1
4,242612,,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation
16,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation
80,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc."
81,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc."
93,Jul 26 2022 8:30AM,242697,10,"Senseonics, Incorporated"
94,Jul 25 2022 4:22PM,242701,15,"Carwin Pharmaceutical Associates, LLC"
95,Jul 25 2022 4:06PM,242695,12,Hush Hush
97,Jul 25 2022 3:18PM,242685,18,Innova Softgel LLC
98,Jul 25 2022 2:39PM,242682,16,Seqirus


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation,,53
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation,,17
2,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation,,12
3,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc.",,3
4,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc.",,8
5,Jul 26 2022 8:30AM,242697,10,"Senseonics, Incorporated",,1
6,Jul 25 2022 4:22PM,242701,15,"Carwin Pharmaceutical Associates, LLC",,1
7,Jul 25 2022 4:06PM,242695,12,Hush Hush,2,
8,Jul 25 2022 3:18PM,242685,18,Innova Softgel LLC,,1
9,Jul 25 2022 2:39PM,242682,16,Seqirus,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation,53,
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation,17,
2,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation,12,
3,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc.",3,
4,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc.",8,
5,Jul 26 2022 8:30AM,242697,10,"Senseonics, Incorporated",1,
6,Jul 25 2022 4:22PM,242701,15,"Carwin Pharmaceutical Associates, LLC",1,
7,Jul 25 2022 4:06PM,242695,12,Hush Hush,,2
8,Jul 25 2022 3:18PM,242685,18,Innova Softgel LLC,1,
9,Jul 25 2022 2:39PM,242682,16,Seqirus,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation,53,
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation,17,
2,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation,12,
3,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc.",3,
4,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc.",8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation,53.0,NaN
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation,17.0,NaN
2,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation,12.0,NaN
3,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc.",3.0,NaN
4,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc.",8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 26 2022 8:30AM,242696,10,ISDIN Corporation,53.0,0.0
1,Jul 26 2022 8:30AM,242699,10,ISDIN Corporation,17.0,0.0
2,Jul 26 2022 8:30AM,242694,10,ISDIN Corporation,12.0,0.0
3,Jul 26 2022 8:30AM,242693,10,"Methapharm, Inc.",3.0,0.0
4,Jul 26 2022 8:30AM,242692,10,"Methapharm, Inc.",8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4610253,165.0,3.0
12,242695,0.0,2.0
15,485333,24.0,40.0
16,485316,2.0,0.0
18,242685,1.0,0.0
19,485344,16.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4610253,165.0,3.0
1,12,242695,0.0,2.0
2,15,485333,24.0,40.0
3,16,485316,2.0,0.0
4,18,242685,1.0,0.0
5,19,485344,16.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,165.0,3.0
1,12,0.0,2.0
2,15,24.0,40.0
3,16,2.0,0.0
4,18,1.0,0.0
5,19,16.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,165.0
1,12,Released,0.0
2,15,Released,24.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19
Status,,,,,,
Executing,3.0,2.0,40.0,0.0,0.0,5.0
Released,165.0,0.0,24.0,2.0,1.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19
0,Executing,3.0,2.0,40.0,0.0,0.0,5.0
1,Released,165.0,0.0,24.0,2.0,1.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19
0,Executing,3.0,2.0,40.0,0.0,0.0,5.0
1,Released,165.0,0.0,24.0,2.0,1.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()